<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1oIwdceg-apNWycn3a4Q52NBWp185hHLi?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




# Day 3 — AI AGENT SERIES

On **Day 2**, we explored how agent teams are made and they collaborate to achive the goal.  

For **Day 3** :<br>
we will make a minimal two-agent team using AutoGen AgentChat:
- **Doer**: Writes and revises code to fix a predefined buggy snippet.
- **Critic**: Reviews the Doer’s output, gives targeted feedback, and says `APPROVE` when the code looks correct.

Flow:
1. Provide a small buggy function.
2. Run a round-robin team until the critic says `APPROVE`.
3. Print the full conversation and the final corrected code.



## Setup and Installation of required libraries
We'll start by installing the prerequisite libraries that we'll be using in this example.

In [1]:
!pip install -q "autogen-agentchat" "autogen-ext[openai,azure]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


## 🧠 What is AutoGen?

AutoGen is a Python library for **conversational multi-agent systems**.  
Each agent can:
- Play a role (e.g., coder, reviewer, user)
- Send/receive natural language messages
- Use tools like code execution, APIs, file I/O
- Collaborate, retry, and refine

Two key agent types:
1. **AssistantAgent** — the "doer" (writes, explains, codes)
2. **UserProxyAgent** — simulates a user (manual or automatic feedback)


## LLM setup
we will setup our llm model using `openai`

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from google.colab import userdata

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=userdata.get('OPENAI_API_KEY'),
)


In [3]:
# Predefined buggy code (simple and intentional mistakes)
buggy_code = r'''
"""Utilities for simple math operations."""

def add_numbers(a, b):
    # BUG: wrong operation used (subtraction instead of addition)
    return a - b

class Divider:
    def __init__(self, numerator: float, denominator: float):
        self.numerator = numerator
        self.denominator = denominator

    def divide(self) -> float:
        # BUG: potential ZeroDivisionError not handled; also wrong attribute name
        return self.numerator / self.denominatr
'''

print("Buggy code:\n", buggy_code)

# Agent system prompts
DOER_SYSTEM = (
    "You are the Doer. Your job is to fix bugs in a given Python code snippet. "
    "Respond with clear reasoning and improved code when needed. Keep changes minimal but correct."
)

CRITIC_SYSTEM = (
    "You are the Critic. Review the Doer’s output, point out specific remaining issues, and say "
    "'APPROVE' ONLY when the code is correct and robust for the simple use cases described."
)


Buggy code:
 
"""Utilities for simple math operations."""

def add_numbers(a, b):
    # BUG: wrong operation used (subtraction instead of addition)
    return a - b

class Divider:
    def __init__(self, numerator: float, denominator: float):
        self.numerator = numerator
        self.denominator = denominator

    def divide(self) -> float:
        # BUG: potential ZeroDivisionError not handled; also wrong attribute name
        return self.numerator / self.denominatr




### 🔍 What happens?
1. The **User** sends the broken code.  
2. The **CodeFixer** suggests a fix (adds zero-check).  
3. The **User** replies: "Thanks, this works now. ✅ fixed".  
4. Chat ends automatically.

✅ No hardcoded if/else. Just natural dialogue between LLMs.


In [5]:
import os
import asyncio
from typing import List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient



## Creating Agents

- Define your Agents, and provide them a `role` and `system prompt`
- It has been seen that LLMs perform better when they are role playing.

In [6]:
# Build the two agents and the team

# Create agents
doer = AssistantAgent(
    "doer",
    model_client=model_client,
    system_message=DOER_SYSTEM,
)

critic = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message=CRITIC_SYSTEM,
)

# Termination: stop when critic says APPROVE
termination_condition = TextMentionTermination("APPROVE")

# Team: simple round-robin chat
team = RoundRobinGroupChat(
    [doer, critic],
    termination_condition=termination_condition,
)

print("Team ready.")


Team ready.


## Running the agents

In [7]:
# Run the team on the debugging task

debug_task = (
    "We have a small buggy Python snippet. Identify the bugs, explain them briefly, "
    "and provide a corrected version. Keep changes minimal. Then await Critic feedback.\n\n"
    f"Buggy code:\n{buggy_code}"
)

async def run_team_once() -> TaskResult:
    await team.reset()
    result: TaskResult = await team.run(task=debug_task)
    return result

# Use asyncio.run if in a script. In notebooks, use nest_asyncio or plain await if supported.
result = await run_team_once()
print("Team finished. Stop reason:", result.stop_reason)


Team finished. Stop reason: Text 'APPROVE' mentioned


In [8]:
# Print full conversation and extract final corrected code
from autogen_agentchat.messages import TextMessage
from typing import Optional

# Pretty-print conversation
for m in result.messages:
    role = getattr(m, "source", "?")
    content = getattr(m, "content", "")
    print(f"===== {role.upper()} =====\n{content}\n")

# Heuristic: find last code block in the Doer messages after APPROVE or overall
import re

def extract_last_code_block(text: str) -> Optional[str]:
    blocks = re.findall(r"```(?:python)?\n([\s\S]*?)```", text)
    if blocks:
        return blocks[-1].strip()
    return None

final_code: Optional[str] = None

# Prefer messages before termination from doer; else scan all
for m in reversed(result.messages):
    if getattr(m, "source", "") == "doer":
        code = extract_last_code_block(getattr(m, "content", ""))
        if code:
            final_code = code
            break

print("\n==== FINAL CORRECTED CODE ====")
print(final_code or "[No code block found; check messages above]")

# Close model client
await model_client.close()


===== USER =====
We have a small buggy Python snippet. Identify the bugs, explain them briefly, and provide a corrected version. Keep changes minimal. Then await Critic feedback.

Buggy code:

"""Utilities for simple math operations."""

def add_numbers(a, b):
    # BUG: wrong operation used (subtraction instead of addition)
    return a - b

class Divider:
    def __init__(self, numerator: float, denominator: float):
        self.numerator = numerator
        self.denominator = denominator

    def divide(self) -> float:
        # BUG: potential ZeroDivisionError not handled; also wrong attribute name
        return self.numerator / self.denominatr


===== DOER =====
Let's address the bugs in the provided code:

1. **Incorrect Operation in `add_numbers` Function:**
   - The function name `add_numbers` suggests that it should perform addition, but it currently performs subtraction (`a - b`).
   
2. **Attribute Typo in `Divider` Class:**
   - In the `divide` method, there is a typo in t


## 📝 Exercises
1. Modify the bug: make it a *list index out of range* error. See if the agent fixes it.  
2. Add a **CriticAgent** that reviews the fix before User accepts.  
3. Try AgentChat: add another tool (like a calculator) and see how the agent uses it.  

---

### 🚀 Next (Day 4)
We’ll explore **Agno Framework** — a lightweight modular system for production-grade agents.
